In [1]:
from dash import Dash, State, html, dash_table, dcc, callback, Output, Input
import pandas as pd


import plotly.graph_objects as go
import numpy as np
from datetime import datetime, timedelta

#import cdstoolbox as ct
#import chart_studio.plotly as py

import chart_studio.plotly as py
import dash_bootstrap_components as dbc




####Figure 2 - caro's mapppp#####

# Read the CSV file
final_daily2 = pd.read_csv('https://raw.githubusercontent.com/BenGoodair/Methane_Dashboard/main/combined_daily.csv')
simulated = pd.read_csv('https://raw.githubusercontent.com/BenGoodair/Methane_Dashboard/main/simulated_data.csv')
# Round 'lon_m' and 'lat_m' columns
final_daily2['lon_m'] = final_daily2['lon_m'].round()
final_daily2['lat_m'] = final_daily2['lat_m'].round()
# Convert 'time' column to datetime
final_daily2['date'] = pd.to_datetime(final_daily2['time']).dt.date
# Group by 'date', 'lon_m', and 'lat_m', and calculate the mean of 'ch4'
mean_ch4 = final_daily2.groupby(['date','lon_m', 'lat_m'])['ch4'].mean().reset_index()
# Remove rows with missing values
mean_ch4 = mean_ch4.dropna()

#convert date to date column
mean_ch4['date'] = pd.to_datetime(mean_ch4['date'])

#order data by date
mean_ch4.sort_values(by='date', ascending=False, inplace=True)
#keep only most recent data for each lon and lat
mean_ch4.drop_duplicates(subset=['lon_m', 'lat_m'], keep='first', inplace=True)
#reset
mean_ch4.reset_index(drop=True, inplace=True)

import plotly.graph_objects as go
from scipy.spatial import cKDTree

import plotly.express as px
import plotly.graph_objects as go

# Create a Plotly choropleth map


import plotly.graph_objects as go
import plotly.express as px
from scipy.spatial.distance import cdist
import numpy as np

hospitals = simulated[simulated['plant_or_hospital'] == "Hospital"]
hospitals = hospitals.rename(columns={'latitude': 'lat'})
hospitals = hospitals.rename(columns={'longitude': 'lon'})

energy_plants = simulated[simulated['plant_or_hospital'] == "Energy plant"]
energy_plants = energy_plants.rename(columns={'latitude': 'lat'})
energy_plants = energy_plants.rename(columns={'longitude': 'lon'})

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from scipy.spatial import cKDTree


df_sorted = mean_ch4.sort_values('ch4', ascending=False)

# Calculate the number of rows representing the top 10%
top_10_percent = int(len(df_sorted) * 0.1)

# Slice the DataFrame to keep only the top rows
top_10_df = df_sorted.head(top_10_percent)

n = len(top_10_df)

# Create a new column 'Ascending' with values from 1 to n
top_10_df['USA_Methane_Ranking'] = range(1, n+1)

top_10_df = top_10_df.rename(columns={'lat_m': 'lat'})
top_10_df = top_10_df.rename(columns={'lon_m': 'lon'})


from scipy.spatial.distance import cdist

# Assuming 'hospitals' and 'top_10_df' are your DataFrames
# Assuming 'hospitals' has columns 'Hospital Name', 'Latitude', and 'Longitude'
# Assuming 'top_10_df' has columns 'Row ID', 'Latitude', and 'Longitude'

# Create a helper function to find the nearest hospital for a given location
def find_nearest_hospital(row, hospitals):
    # Create a DataFrame with coordinates of the current row
    coords = pd.DataFrame({'lat': [row['lat']], 'lon': [row['lon']]})

    # Calculate the distances between the coordinates of the current row and all hospitals
    distances = cdist(coords.values, hospitals[['lat', 'lon']].values, metric='euclidean')

    # Find the index of the nearest hospital
    nearest_index = distances.argmin()

    # Return the name of the nearest hospital
    return hospitals.iloc[nearest_index]['company_name']

# Apply the helper function to each row in 'top_10_df'
top_10_df['Nearest_Hospital'] = top_10_df.apply(lambda row: find_nearest_hospital(row, hospitals), axis=1)



# Create a helper function to find the nearest hospital for a given location
def find_nearest_plant(row, energy_plants):
    # Create a DataFrame with coordinates of the current row
    coords = pd.DataFrame({'lat': [row['lat']], 'lon': [row['lon']]})

    # Calculate the distances between the coordinates of the current row and all hospitals
    distances = cdist(coords.values, energy_plants[['lat', 'lon']].values, metric='euclidean')


    # Find the index of the nearest hospital
    nearest_index = distances.argmin()

    # Return the name of the nearest hospital
    return energy_plants.iloc[nearest_index]['company_name']

# Apply the helper function to each row in 'top_10_df'
top_10_df['Nearest_Energy_plant'] = top_10_df.apply(lambda row: find_nearest_plant(row, energy_plants), axis=1)





min_value = top_10_df['ch4'].min()
max_value = top_10_df['ch4'].max()
scaled_values = ((top_10_df['ch4'] - min_value) / (max_value - min_value)) ** 1.2 * 15

customdata = top_10_df[['USA_Methane_Ranking', 'Nearest_Hospital', 'Nearest_Energy_plant', 'ch4']].values.tolist()
customdata = np.stack((top_10_df['USA_Methane_Ranking'], top_10_df['Nearest_Hospital'], top_10_df['Nearest_Energy_plant'], top_10_df['ch4']), axis=-1)


# Create the scatter plot with scaled point sizes
tab1_fig2 = px.scatter_mapbox(top_10_df, lat="lat", lon="lon", hover_name="ch4",
                         size=scaled_values,
                         color_continuous_scale='viridis',
                         mapbox_style="carto-positron",
                         zoom=2,
                         center={'lat': 39.8283, 'lon': -98.5795},
                         opacity=0.8, size_max=30,    
                         custom_data=['USA_Methane_Ranking', 'Nearest_Hospital', 'Nearest_Energy_plant', 'ch4'])


tab1_fig2.update_traces(hovertemplate='Methane ranking: %{customdata[0]}<br>Nearest Hospital: %{customdata[1]}<br>Nearest Energy Plant: %{customdata[2]}<br>Methane particles (Mole fraction): %{customdata[3]}')



/var/folders/9x/mzy5y7vn06s46r3f60xxy7qw0000gp/T/ipykernel_49240/502458037.py:21: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  final_daily2 = pd.read_csv('https://raw.githubusercontent.com/BenGoodair/Methane_Dashboard/main/combined_daily.csv')
/var/folders/9x/mzy5y7vn06s46r3f60xxy7qw0000gp/T/ipykernel_49240/502458037.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_10_df['USA_Methane_Ranking'] = range(1, n+1)


In [6]:
import dash 
from dash import dash_table
from dash import State
import reverse_geocoder as rg
from dash.dependencies import Input, Output, State, MATCH



app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
       dcc.Graph(
        id='map'
    )
])


def find_nearest_point(lat, lon, df):
    point = np.array([[lat, lon]])
    distances = cdist(point, df[['lat', 'lon']])
    nearest_index = np.argmin(distances)
    nearest_point = df.iloc[nearest_index]
    return nearest_point



# Define the callback function to update the scatter plot based on the dropdown selection

@app.callback(
    Output('map', 'figure'), 
    [Input('map', 'clickData')])

def update_nearest_hospital(click_data):
    if click_data:
        point = click_data['points'][0]

        lat = point['lat']
        lon = point['lon']

        nearest_hospital = find_nearest_point(lat, lon, hospitals)
        nearest_energy_plant = find_nearest_point(lat, lon, energy_plants)

        # Create a new figure with the nearest hospital and energy plant annotations
        new_fig = go.Figure(tab1_fig2)

        new_fig.add_trace(
            go.Scattermapbox(
                lat=[lat, nearest_hospital['lat']],
                lon=[lon, nearest_hospital['lon']],
                mode='lines',
                line=dict(color='red', width=2),
                hoverinfo='none'
            )
        )
        new_fig.add_trace(
            go.Scattermapbox(
                lat=[lat, nearest_energy_plant['lat']],
                lon=[lon, nearest_energy_plant['lon']],
                mode='lines',
                line=dict(color='blue', width=2),
                hoverinfo='none'
            )
        )
        new_fig.add_trace(
            go.Scattermapbox(
                lat=[nearest_hospital['lat']],
                lon=[nearest_hospital['lon']],
                mode='markers',
                customdata=np.dstack((nearest_hospital["company_name"], nearest_hospital["phone_number"], nearest_hospital["number_of_employees"], nearest_hospital["previous_leaks_n"], nearest_hospital["fossil_fuel_type"], nearest_hospital["number_of_beds"])),
                marker=dict(size=10, color='red'),
                hovertemplate='Company Name: %{customdata[0][0]}<br>Company telephone: %{customdata[0][1]}<br>Number of Employees: %{customdata[0][2]}<br>Number of Previous Leaks: %{customdata[0][3]}<br>Fossil Fuel Type: %{customdata[0][4]}<br>Number of Hospital Beds: %{customdata[0][5]}'
            )
        )
        new_fig.add_trace(
            go.Scattermapbox(
                lat=[nearest_energy_plant['lat']],
                lon=[nearest_energy_plant['lon']],
                mode='markers',
                customdata=np.dstack((nearest_energy_plant["company_name"], nearest_energy_plant["phone_number"], nearest_energy_plant["number_of_employees"], nearest_energy_plant["previous_leaks_n"], nearest_energy_plant["fossil_fuel_type"], nearest_energy_plant["number_of_beds"])),
                marker=dict(size=10, color='blue'),
                hovertemplate='Company Name: %{customdata[0][0]}<br>Company telephone: %{customdata[0][1]}<br>Number of Employees: %{customdata[0][2]}<br>Number of Previous Leaks: %{customdata[0][3]}<br>Fossil Fuel Type: %{customdata[0][4]}<br>Number of Hospital Beds: %{customdata[0][5]}'
            )   
        )
       # Set the zoom and view location from the previous figure
        new_fig.update_layout(mapbox=dict(center=dict(lat=tab1_fig2['layout']['mapbox']['center']['lat'],
                                                      lon=tab1_fig2['layout']['mapbox']['center']['lon']),
                                               zoom=tab1_fig2['layout']['mapbox']['zoom']))

        return new_fig

    return tab1_fig2


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8001)

